## 1. Xử lý thông tin các địa chỉ IP gửi request tới hệ thống
Lưu lại thông tin địa chỉ IP và số lần gửi request tương ứng dưới dạng dictionary

In [ ]:
# Import thư viện regular expression
import re

# Lưu địa chỉ file log vào file_path
file_path = 'modsec_audit.log'
# Pattern của biểu thức chính quy
pattern = r'--\w+-A--\n\[[\w@ \/\:\-]+\] [\w@]+ \b((?:\d{1,3}\.){3}\d{1,3})\b'

# Khởi tạo biến dict ip_count
ip_count = {}

In [ ]:
# Đọc thông tin từ file log theo cụm (mỗi cụm cách nhau bởi 1 dòng trống)
with open(file_path, 'r', encoding='latin-1') as file:
    log_text = ''
    line = file.readline()
    while line:
        if line.strip() != '':  # Kiểm tra nếu dòng không phải là dòng trống
            log_text += line
        elif log_text != '':
            matches = re.findall(pattern, log_text)
            for ip in matches:
                if ip in ip_count:
                    ip_count[ip] += 1
                else:
                    ip_count[ip] = 1
            log_text = ''
        line = file.readline()

# Kiểm tra xem có đoạn văn bản cuối cùng cần được xử lý không
if log_text != '':
    matches = re.findall(pattern, log_text)
    for ip in matches:
        if ip in ip_count:
            ip_count[ip] += 1
        else:
            ip_count[ip] = 1

# Hoàn thành lưu trữ thông tin IP và số lần gửi request tương ứng tới hệ thống trong ip_count
# print(ip_count)

## 2. Chuyển từ địa chỉ IP chuyển thành location (country)

In [ ]:
# Import thư viện chuyển từ IP sang country_code
import IP2Location

# Hàm xử lý chuyển từ IP sang country_code
def get_country_code(ip_address):
    database = IP2Location.IP2Location('IPV6-COUNTRY.SAMPLE.BIN')
    result = database.get_all(ip_address)
    country_code = result.country_short
    return country_code

# Dict chứa country_code và số request tương ứng
location = {}

# Duyệt qua tất cả ip_address trong ip_count và lưu trữ trong dict location
for ip_address in ip_count:
    country_code = get_country_code(ip_address)
    if country_code in location:
        location[country_code] += ip_count[ip_address]
    else: 
        location[country_code] = ip_count[ip_address]

# print(location)


## 3. Vẽ biểu đồ

In [ ]:
# Thư viện xử lý vẽ biểu đồ
import matplotlib.pyplot as plt

# Minimum counts cho các quốc gia trong biểu đồ
MINIMUM = 1000

3.1 Biểu đồ tròn

In [ ]:
# Tách danh sách các quốc gia và số lượng thành hai danh sách riêng
countries = list(location.keys())
counts = list(location.values())

# Tìm các quốc gia có giá trị < MINIMUM
small_countries = [country for country, count in zip(countries, counts) if count < MINIMUM]
small_counts = [count for count in counts if count < MINIMUM]

# Tính tổng số lượng các quốc gia nhỏ (nhỏ hơn MINIMUM)
sum_small_counts = 0
for small_count in small_counts:
    sum_small_counts += small_count

# Thêm mục "Khác" vào danh sách quốc gia và số lượng
countries.append('Khác')
counts.append(sum_small_counts)

# Loại bỏ các quốc gia nhỏ từ danh sách ban đầu
countries = [country for country, count in zip(countries, counts) if count >= MINIMUM]
counts = [count for count in counts if count >= MINIMUM]

# Vẽ biểu đồ tròn
plt.pie(counts, labels=countries, autopct='%1.1f%%')

# Đặt tiêu đề cho biểu đồ
plt.title('IP Addresses by Country')

# Hiển thị biểu đồ
plt.show()


3.2 Biểu đồ cột

In [ ]:
# Tách danh sách các quốc gia và số lượng thành hai danh sách riêng
countries = list(location.keys())
counts = list(location.values())

# Tạo danh sách chỉ số và danh sách các nước thỏa mãn điều kiện
filtered_indexes = [i for i, count in enumerate(counts) if count >= MINIMUM]
filtered_countries = [countries[i] for i in filtered_indexes]
filtered_counts = [counts[i] for i in filtered_indexes]

# Tạo một list các chỉ số từ 0 đến độ dài của danh sách nước thỏa mãn điều kiện
x_indexes = range(len(filtered_countries))

# Vẽ biểu đồ cột
plt.bar(x_indexes, filtered_counts)

# Đặt nhãn cho trục x
plt.xticks(x_indexes, filtered_countries, rotation='vertical')

# Đặt tiêu đề cho biểu đồ
plt.title('IP Addresses by Country (Counts >= {})'.format(MINIMUM))

# Hiển thị biểu đồ
plt.show()
